# Tensor builder walkthrough


PixelPrism exposes the variadic ``build_tensor`` operator to assemble tensors from scalar math expressions. This notebook highlights how constants, variables, and derived expressions can be interleaved while targeting vectors or higher-rank arrays.


In [1]:

import pixelprism.math as pm
import pixelprism.math.functional as F

scalar_alpha = pm.const("builder_scalar_alpha", data=1.5, dtype=pm.DType.FLOAT32)
scalar_beta = pm.const("builder_scalar_beta", data=-2.0, dtype=pm.DType.FLOAT64)
scalar_gamma = pm.const("builder_scalar_gamma", data=3, dtype=pm.DType.INT32)

builder_variable = pm.var("builder_variable", dtype=pm.DType.FLOAT32, shape=())
pm.set_value(builder_variable.name, 0.25)

def inspect(expr):
    tensor = expr.eval()
    print(f"{expr.name}: shape={tensor.shape.dims}, dtype={tensor.dtype}")
    print(tensor.value)
    return tensor


### Building simple vectors
The operator defaults to a 1-D tensor whose length matches the number of provided elements.

In [2]:
basic_vector = F.build_tensor([scalar_alpha, scalar_beta, 7])
inspect(basic_vector)


tensor([builder_scalar_alpha, builder_scalar_beta, constant_4]): shape=(3,), dtype=DType.FLOAT64
[ 1.5 -2.   7. ]


tensor([1.5, -2.0, 7.0], dtype=DType.FLOAT64, shape=(3,), mutable=True)

### Mixing variables and expressions
Every entry can be a variable or a composed expression. Updating the context immediately impacts the built tensor.

In [3]:
pm.set_value(builder_variable.name, 1.75)
expression_vector = F.build_tensor(
    [
        builder_variable,
        scalar_alpha + scalar_beta,
        F.mul(builder_variable, scalar_gamma),
        scalar_gamma + 0.5,
    ]
)
inspect(expression_vector)


tensor([builder_variable, builder_scalar_alpha + builder_scalar_beta, builder_variable * builder_scalar_gamma, builder_scalar_gamma + constant_8]): shape=(4,), dtype=DType.FLOAT64
[ 1.75 -0.5   5.25  3.5 ]


tensor([1.75, -0.5, 5.25, 3.5], dtype=DType.FLOAT64, shape=(4,), mutable=True)

### Multi-dimensional tensors
Passing ``shape`` reshapes the flat buffer, enabling matrices or higher-order tensors.

In [4]:
pm.set_value(builder_variable.name, -0.25)
matrix_example = F.build_tensor(
    [
        scalar_alpha,
        builder_variable,
        scalar_beta,
        builder_variable + scalar_gamma,
    ],
    shape=(2, 2),
)
inspect(matrix_example)


tensor([builder_scalar_alpha, builder_variable, builder_scalar_beta, builder_variable + builder_scalar_gamma]) @ (2, 2): shape=(2, 2), dtype=DType.FLOAT64
[[ 1.5  -0.25]
 [-2.    2.75]]


tensor([[1.5, -0.25], [-2.0, 2.75]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

### Larger layouts and dtype promotion
Shapes of any rank are supported so long as the flat element count matches. Mixed dtypes automatically promote to the highest precision involved.

In [5]:
pm.set_value(builder_variable.name, 0.0)
volume = F.build_tensor(
    [
        scalar_alpha,
        builder_variable,
        scalar_beta,
        scalar_gamma,
        builder_variable + scalar_gamma,
        scalar_alpha - scalar_beta,
        0.0,
        builder_variable - 1.0,
    ],
    shape=(2, 2, 2),
)
inspect(volume)


tensor([builder_scalar_alpha, builder_variable, builder_scalar_beta, builder_scalar_gamma, builder_variable + builder_scalar_gamma, builder_scalar_alpha - builder_scalar_beta, constant_17, builder_variable - constant_15]) @ (2, 2, 2): shape=(2, 2, 2), dtype=DType.FLOAT64
[[[ 1.5  0. ]
  [-2.   3. ]]

 [[ 3.   3.5]
  [ 0.  -1. ]]]


tensor([[[1.5, 0.0], [-2.0, 3.0]], [[3.0, 3.5], [0.0, -1.0]]], dtype=DType.FLOAT64, shape=(2, 2, 2), mutable=True)